In [49]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import re
import nbconvert
import time
import os
import numpy as np
from secret import username,password
import psycopg2
from sqlalchemy import create_engine

In [18]:
# Activate splinter
executable_path = {'executable_path': r'C:\Users\nvora\AppData\Roaming\chromedriver_win32\chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:
# Set Web Scrape URL

unemployment_url = "https://data.ers.usda.gov/reports.aspx?ID=17828"

In [3]:
# Scrape Data for each State to acquire county level data - Download each file as a CSV
### Note - I tried to scrape the data directly from the HTML itself but the URL for this site does not change when you query different data 
#### Using Splinter I was at least able to scrape all the data
list_of_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
                  'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
                  'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 
                  'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 
                  'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
                  'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 
                  'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 
                  'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [21]:
# Use Splinter to click through each State and download the associated data to Excel

for state in list_of_states:
    browser.visit(unemployment_url)
    time.sleep(6)
    browser.click_link_by_partial_text(state)
    time.sleep(3)
    browser.click_link_by_id('ctl00_MainContentPlaceHolder_reportingServicesWrapper1__reportViewer_ctl05_ctl04_ctl00')
    time.sleep(2)
    browser.click_link_by_partial_text('Excel')
    time.sleep(1)

C:\Users\nvora\Anaconda3\envs\PythonData\lib\site-packages\splinter-0.14.0-py3.8.egg\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.


In [4]:
# Read all of the scraped excel files into a list of dataframes

list_of_dfs = []

for x in range(len(list_of_states)):
    filepath = os.path.join(f'Data_Files/UnemploymentReport ({x}).xlsx')
    state_df = pd.read_excel(filepath,header=1)
    list_of_dfs.append(state_df)

In [5]:
# Concatenate the list of dataframes into a single dataframe with all states and counties

unemployment_df = pd.concat(list_of_dfs)

unemployment_df

,FIPS,Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,Median Household Income (2018),% of State Median HH Income
0,01000,Alabama,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9,3.0,49881.0,1.000000
1,01001,"Autauga County, AL",8.4,6.9,6.2,5.8,5.2,5.1,3.9,3.6,2.7,59338.0,1.189591
2,01003,"Baldwin County, AL",9.0,7.5,6.6,6.1,5.5,5.3,4.1,3.6,2.7,57588.0,1.154508
3,01005,"Barbour County, AL",11.5,11.5,10.2,10.5,8.9,8.3,5.8,5.1,3.8,34382.0,0.689280
4,01007,"Bibb County, AL",10.5,8.5,7.9,7.2,6.6,6.4,4.4,3.9,3.1,46064.0,0.923478
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,56041,"Uinta County, WY",6.2,5.6,5.3,4.8,5.1,5.8,4.5,4.2,3.9,63401.0,1.018343
22,56043,"Washakie County, WY",5.6,5.2,5.0,4.3,4.2,4.8,4.0,4.1,3.9,55190.0,0.886458
23,56045,"Weston County, WY",4.6,4.1,3.6,3.3,3.4,5.0,3.9,3.3,2.9,54319.0,0.872468
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## DATA CLEANING


In [7]:
# Strip the leading 0 in FIPS code and rename FIPS to remove empty space

unemployment_df['FIPS '] = unemployment_df['FIPS '].astype(str) 


unemployment_df['FIPS '] = [fips.lstrip('0') for fips in unemployment_df['FIPS ']]

unemployment_df = unemployment_df.rename(columns={'FIPS ':'FIPS'})

unemployment_df

,FIPS,Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,Median Household Income (2018),% of State Median HH Income
0,1000,Alabama,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9,3.0,49881.0,1.000000
1,1001,"Autauga County, AL",8.4,6.9,6.2,5.8,5.2,5.1,3.9,3.6,2.7,59338.0,1.189591
2,1003,"Baldwin County, AL",9.0,7.5,6.6,6.1,5.5,5.3,4.1,3.6,2.7,57588.0,1.154508
3,1005,"Barbour County, AL",11.5,11.5,10.2,10.5,8.9,8.3,5.8,5.1,3.8,34382.0,0.689280
4,1007,"Bibb County, AL",10.5,8.5,7.9,7.2,6.6,6.4,4.4,3.9,3.1,46064.0,0.923478
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,56041,"Uinta County, WY",6.2,5.6,5.3,4.8,5.1,5.8,4.5,4.2,3.9,63401.0,1.018343
22,56043,"Washakie County, WY",5.6,5.2,5.0,4.3,4.2,4.8,4.0,4.1,3.9,55190.0,0.886458
23,56045,"Weston County, WY",4.6,4.1,3.6,3.3,3.4,5.0,3.9,3.3,2.9,54319.0,0.872468
24,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## Separate the data into two DataFrames - one with State level data, and one with County Level Data
# County DataFrame

county_unemployment_df = unemployment_df[~unemployment_df['Name'].isin(list_of_states)].dropna()

county_unemployment_df


,FIPS,Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,Median Household Income (2018),% of State Median HH Income
1,1001,"Autauga County, AL",8.4,6.9,6.2,5.8,5.2,5.1,3.9,3.6,2.7,59338.0,1.189591
2,1003,"Baldwin County, AL",9.0,7.5,6.6,6.1,5.5,5.3,4.1,3.6,2.7,57588.0,1.154508
3,1005,"Barbour County, AL",11.5,11.5,10.2,10.5,8.9,8.3,5.8,5.1,3.8,34382.0,0.689280
4,1007,"Bibb County, AL",10.5,8.5,7.9,7.2,6.6,6.4,4.4,3.9,3.1,46064.0,0.923478
5,1009,"Blount County, AL",8.7,6.9,6.3,6.1,5.4,5.4,4.0,3.5,2.7,50412.0,1.010645
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,56037,"Sweetwater County, WY",5.3,4.8,4.4,4.0,4.8,6.1,4.5,4.0,3.9,73315.0,1.177581
20,56039,"Teton County, WY",7.5,6.7,5.6,4.6,3.9,3.5,2.8,2.9,2.7,99087.0,1.591529
21,56041,"Uinta County, WY",6.2,5.6,5.3,4.8,5.1,5.8,4.5,4.2,3.9,63401.0,1.018343
22,56043,"Washakie County, WY",5.6,5.2,5.0,4.3,4.2,4.8,4.0,4.1,3.9,55190.0,0.886458


In [9]:
# Remove the State abbreviations from the name to match the county names with the other tables

county_unemployment_df['County'] = county_unemployment_df['Name'].str.split(',').str[0].drop(columns='Name',axis=1)

county_unemployment_df['Name'] = county_unemployment_df['County']

county_unemployment_df = county_unemployment_df.drop(columns='County',axis=1).rename(columns={'Name':'county'})

county_unemployment_df

,FIPS,county,2011,2012,2013,2014,2015,2016,2017,2018,2019,Median Household Income (2018),% of State Median HH Income
1,1001,Autauga County,8.4,6.9,6.2,5.8,5.2,5.1,3.9,3.6,2.7,59338.0,1.189591
2,1003,Baldwin County,9.0,7.5,6.6,6.1,5.5,5.3,4.1,3.6,2.7,57588.0,1.154508
3,1005,Barbour County,11.5,11.5,10.2,10.5,8.9,8.3,5.8,5.1,3.8,34382.0,0.689280
4,1007,Bibb County,10.5,8.5,7.9,7.2,6.6,6.4,4.4,3.9,3.1,46064.0,0.923478
5,1009,Blount County,8.7,6.9,6.3,6.1,5.4,5.4,4.0,3.5,2.7,50412.0,1.010645
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,56037,Sweetwater County,5.3,4.8,4.4,4.0,4.8,6.1,4.5,4.0,3.9,73315.0,1.177581
20,56039,Teton County,7.5,6.7,5.6,4.6,3.9,3.5,2.8,2.9,2.7,99087.0,1.591529
21,56041,Uinta County,6.2,5.6,5.3,4.8,5.1,5.8,4.5,4.2,3.9,63401.0,1.018343
22,56043,Washakie County,5.6,5.2,5.0,4.3,4.2,4.8,4.0,4.1,3.9,55190.0,0.886458


In [10]:
# Narrow Down Columns to relevant/more recent data & rename Unemployment Column

county_unemployment_df = county_unemployment_df[['FIPS','county','2019','Median Household Income (2018)','% of State Median HH Income']]

county_unemployment_df = county_unemployment_df.rename(columns={'2019':'Latest Annual Unemployment Rate (2019)'})

In [11]:
county_unemployment_df.head()

,FIPS,county,Latest Annual Unemployment Rate (2019),Median Household Income (2018),% of State Median HH Income
1,1001,Autauga County,2.7,59338.0,1.189591
2,1003,Baldwin County,2.7,57588.0,1.154508
3,1005,Barbour County,3.8,34382.0,0.689280
4,1007,Bibb County,3.1,46064.0,0.923478
5,1009,Blount County,2.7,50412.0,1.010645


In [12]:
## Separate the data into two DataFrames - one with State level data, and one with County Level Data
# State Dataframe
state_unemployment_df = unemployment_df[unemployment_df['Name'].isin(list_of_states)]

# District of Columbia appeared twice in this dataframe because the county name is also the state name
state_unemployment_df = state_unemployment_df.drop_duplicates('Name')


In [13]:
# Narrow down columns and rename columns like above

state_unemployment_df = state_unemployment_df[['FIPS','Name','2019','Median Household Income (2018)','% of State Median HH Income']].rename(columns={'Name':'State','2019':'Latest Annual Unemployment Rate (2019)'})

In [14]:
state_unemployment_df.head()

,FIPS,State,Latest Annual Unemployment Rate (2019),Median Household Income (2018),% of State Median HH Income
0,1000,Alabama,3.0,49881.0,1.0
0,2000,Alaska,6.1,74912.0,1.0
0,4000,Arizona,4.7,59079.0,1.0
0,5000,Arkansas,3.5,47094.0,1.0
0,6000,California,4.0,75250.0,1.0


In [95]:
# Read in county voter data

##    Had to set dtype as str - the file was loading all numbers (including FIPS) with .0 attached
voter_df = pd.read_csv('Data_Files/countypres_2000-2016.csv',dtype=str)

voter_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942,17208,20191203
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993,17208,20191203
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160,17208,20191203
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113,17208,20191203
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997,56480,20191203


In [96]:
## DATA CLEANING

In [97]:
# Narrow Voter Data to most recent Data

voter_df_2016 = voter_df.loc[voter_df['year']=='2016',:]

# FIll NaN values in party column with Other

voter_df_2016['party'] = voter_df_2016['party'].fillna('Other')

voter_df_2016.head(10)

<ipython-input-97-1e1d4dea5a9f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voter_df_2016['party'] = voter_df_2016['party'].fillna('Other')


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
40517,2016,Alabama,AL,Autauga,1001,President,Hillary Clinton,democrat,5936,24973,20191203
40518,2016,Alabama,AL,Autauga,1001,President,Donald Trump,republican,18172,24973,20191203
40519,2016,Alabama,AL,Autauga,1001,President,Other,Other,865,24973,20191203
40520,2016,Alabama,AL,Baldwin,1003,President,Hillary Clinton,democrat,18458,95215,20191203
40521,2016,Alabama,AL,Baldwin,1003,President,Donald Trump,republican,72883,95215,20191203
40522,2016,Alabama,AL,Baldwin,1003,President,Other,Other,3874,95215,20191203
40523,2016,Alabama,AL,Barbour,1005,President,Hillary Clinton,democrat,4871,10469,20191203
40524,2016,Alabama,AL,Barbour,1005,President,Donald Trump,republican,5454,10469,20191203
40525,2016,Alabama,AL,Barbour,1005,President,Other,Other,144,10469,20191203
40526,2016,Alabama,AL,Bibb,1007,President,Hillary Clinton,democrat,1874,8819,20191203


In [98]:
# Remove unnecessary columns

voter_df2 = voter_df_2016[['state','county','FIPS','office','party','candidatevotes']]

voter_df2.head(10)

,state,county,FIPS,office,party,candidatevotes
40517,Alabama,Autauga,1001,President,democrat,5936
40518,Alabama,Autauga,1001,President,republican,18172
40519,Alabama,Autauga,1001,President,Other,865
40520,Alabama,Baldwin,1003,President,democrat,18458
40521,Alabama,Baldwin,1003,President,republican,72883
40522,Alabama,Baldwin,1003,President,Other,3874
40523,Alabama,Barbour,1005,President,democrat,4871
40524,Alabama,Barbour,1005,President,republican,5454
40525,Alabama,Barbour,1005,President,Other,144
40526,Alabama,Bibb,1007,President,democrat,1874


In [99]:
voter_df3 = voter_df2[['county','party','candidatevotes']]

voter_df3.head(10)

,county,party,candidatevotes
40517,Autauga,democrat,5936
40518,Autauga,republican,18172
40519,Autauga,Other,865
40520,Baldwin,democrat,18458
40521,Baldwin,republican,72883
40522,Baldwin,Other,3874
40523,Barbour,democrat,4871
40524,Barbour,republican,5454
40525,Barbour,Other,144
40526,Bibb,democrat,1874


In [100]:
### Rearrange the dataframe to only have a single county row

democrat_df = pd.merge(voter_df_2016['FIPS'],voter_df2.loc[voter_df3['party'] == 'democrat',:],on='FIPS',how='right')

democrat_df = democrat_df.drop_duplicates()

democrat_df.head(10)

,FIPS,state,county,office,party,candidatevotes
0,1001,Alabama,Autauga,President,democrat,5936
3,1003,Alabama,Baldwin,President,democrat,18458
6,1005,Alabama,Barbour,President,democrat,4871
9,1007,Alabama,Bibb,President,democrat,1874
12,1009,Alabama,Blount,President,democrat,2156
15,1011,Alabama,Bullock,President,democrat,3530
18,1013,Alabama,Butler,President,democrat,3726
21,1015,Alabama,Calhoun,President,democrat,13242
24,1017,Alabama,Chambers,President,democrat,5784
27,1019,Alabama,Cherokee,President,democrat,1547


In [101]:
democrat_republican_df = pd.merge(democrat_df,voter_df2.loc[voter_df2['party'] == 'republican',:],on=['FIPS'],how='right')

democrat_republican_df = democrat_republican_df.drop_duplicates()

democrat_republican_df.head(15)

,FIPS,state_x,county_x,office_x,party_x,candidatevotes_x,state_y,county_y,office_y,party_y,candidatevotes_y
0,1001,Alabama,Autauga,President,democrat,5936,Alabama,Autauga,President,republican,18172
1,1003,Alabama,Baldwin,President,democrat,18458,Alabama,Baldwin,President,republican,72883
2,1005,Alabama,Barbour,President,democrat,4871,Alabama,Barbour,President,republican,5454
3,1007,Alabama,Bibb,President,democrat,1874,Alabama,Bibb,President,republican,6738
4,1009,Alabama,Blount,President,democrat,2156,Alabama,Blount,President,republican,22859
5,1011,Alabama,Bullock,President,democrat,3530,Alabama,Bullock,President,republican,1140
6,1013,Alabama,Butler,President,democrat,3726,Alabama,Butler,President,republican,4901
7,1015,Alabama,Calhoun,President,democrat,13242,Alabama,Calhoun,President,republican,32865
8,1017,Alabama,Chambers,President,democrat,5784,Alabama,Chambers,President,republican,7843
9,1019,Alabama,Cherokee,President,democrat,1547,Alabama,Cherokee,President,republican,8953


In [102]:
all_parties_df = pd.merge(democrat_republican_df,voter_df2.loc[voter_df3['party']== 'Other',:],on='FIPS',how='right')

all_parties_df

,FIPS,state_x,county_x,office_x,party_x,candidatevotes_x,state_y,county_y,office_y,party_y,candidatevotes_y,state,county,office,party,candidatevotes
0,1001,Alabama,Autauga,President,democrat,5936,Alabama,Autauga,President,republican,18172,Alabama,Autauga,President,Other,865
1,1003,Alabama,Baldwin,President,democrat,18458,Alabama,Baldwin,President,republican,72883,Alabama,Baldwin,President,Other,3874
2,1005,Alabama,Barbour,President,democrat,4871,Alabama,Barbour,President,republican,5454,Alabama,Barbour,President,Other,144
3,1007,Alabama,Bibb,President,democrat,1874,Alabama,Bibb,President,republican,6738,Alabama,Bibb,President,Other,207
4,1009,Alabama,Blount,President,democrat,2156,Alabama,Blount,President,republican,22859,Alabama,Blount,President,Other,573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,2037,Alaska,District 37,President,democrat,2421,Alaska,District 37,President,republican,1938,Alaska,District 37,President,Other,703
3178,2038,Alaska,District 38,President,democrat,2758,Alaska,District 38,President,republican,1143,Alaska,District 38,President,Other,1194
3179,2039,Alaska,District 39,President,democrat,3142,Alaska,District 39,President,republican,1405,Alaska,District 39,President,Other,1092
3180,2040,Alaska,District 40,President,democrat,2338,Alaska,District 40,President,republican,1377,Alaska,District 40,President,Other,895


In [103]:
# Drop duplicate rows, narrow columns, and rename columns

all_parties_df = all_parties_df.drop_duplicates()

all_parties_df

,FIPS,state_x,county_x,office_x,party_x,candidatevotes_x,state_y,county_y,office_y,party_y,candidatevotes_y,state,county,office,party,candidatevotes
0,1001,Alabama,Autauga,President,democrat,5936,Alabama,Autauga,President,republican,18172,Alabama,Autauga,President,Other,865
1,1003,Alabama,Baldwin,President,democrat,18458,Alabama,Baldwin,President,republican,72883,Alabama,Baldwin,President,Other,3874
2,1005,Alabama,Barbour,President,democrat,4871,Alabama,Barbour,President,republican,5454,Alabama,Barbour,President,Other,144
3,1007,Alabama,Bibb,President,democrat,1874,Alabama,Bibb,President,republican,6738,Alabama,Bibb,President,Other,207
4,1009,Alabama,Blount,President,democrat,2156,Alabama,Blount,President,republican,22859,Alabama,Blount,President,Other,573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,2037,Alaska,District 37,President,democrat,2421,Alaska,District 37,President,republican,1938,Alaska,District 37,President,Other,703
3178,2038,Alaska,District 38,President,democrat,2758,Alaska,District 38,President,republican,1143,Alaska,District 38,President,Other,1194
3179,2039,Alaska,District 39,President,democrat,3142,Alaska,District 39,President,republican,1405,Alaska,District 39,President,Other,1092
3180,2040,Alaska,District 40,President,democrat,2338,Alaska,District 40,President,republican,1377,Alaska,District 40,President,Other,895


In [104]:
# Clean all_parties_df

all_parties_cleaned = all_parties_df[['FIPS','state_x','county_x','office_x','candidatevotes_x','candidatevotes_y','candidatevotes']]



In [105]:
all_parties_cleaned2 = all_parties_cleaned.rename(columns = {'state_x':'state','county_x':'county','office_x':'office','candidatevotes_x':'Democrat Votes','candidatevotes_y':'Republican Votes','candidatevotes':'Other Votes'})


all_parties_cleaned2

,FIPS,state,county,office,Democrat Votes,Republican Votes,Other Votes
0,1001,Alabama,Autauga,President,5936,18172,865
1,1003,Alabama,Baldwin,President,18458,72883,3874
2,1005,Alabama,Barbour,President,4871,5454,144
3,1007,Alabama,Bibb,President,1874,6738,207
4,1009,Alabama,Blount,President,2156,22859,573
...,...,...,...,...,...,...,...
3177,2037,Alaska,District 37,President,2421,1938,703
3178,2038,Alaska,District 38,President,2758,1143,1194
3179,2039,Alaska,District 39,President,3142,1405,1092
3180,2040,Alaska,District 40,President,2338,1377,895


In [106]:
#### Final Tables for Load Process (Overall Summary)
## County Unemployment DataFrame

county_unemployment_df


,FIPS,county,Latest Annual Unemployment Rate (2019),Median Household Income (2018),% of State Median HH Income
1,1001,Autauga County,2.7,59338.0,1.189591
2,1003,Baldwin County,2.7,57588.0,1.154508
3,1005,Barbour County,3.8,34382.0,0.689280
4,1007,Bibb County,3.1,46064.0,0.923478
5,1009,Blount County,2.7,50412.0,1.010645
...,...,...,...,...,...
19,56037,Sweetwater County,3.9,73315.0,1.177581
20,56039,Teton County,2.7,99087.0,1.591529
21,56041,Uinta County,3.9,63401.0,1.018343
22,56043,Washakie County,3.9,55190.0,0.886458


In [107]:
## State Unemployment DataFrame

state_unemployment_df


,FIPS,State,Latest Annual Unemployment Rate (2019),Median Household Income (2018),% of State Median HH Income
0,1000,Alabama,3.0,49881.0,1.0
0,2000,Alaska,6.1,74912.0,1.0
0,4000,Arizona,4.7,59079.0,1.0
0,5000,Arkansas,3.5,47094.0,1.0
0,6000,California,4.0,75250.0,1.0
0,8000,Colorado,2.8,71949.0,1.0
0,9000,Connecticut,3.7,76366.0,1.0
0,10000,Delaware,3.8,65467.0,1.0
0,11000,District of Columbia,5.5,82533.0,1.0
0,12000,Florida,3.1,55433.0,1.0


In [108]:
## Voter breakdown by county

all_parties_cleaned2

,FIPS,state,county,office,Democrat Votes,Republican Votes,Other Votes
0,1001,Alabama,Autauga,President,5936,18172,865
1,1003,Alabama,Baldwin,President,18458,72883,3874
2,1005,Alabama,Barbour,President,4871,5454,144
3,1007,Alabama,Bibb,President,1874,6738,207
4,1009,Alabama,Blount,President,2156,22859,573
...,...,...,...,...,...,...,...
3177,2037,Alaska,District 37,President,2421,1938,703
3178,2038,Alaska,District 38,President,2758,1143,1194
3179,2039,Alaska,District 39,President,3142,1405,1092
3180,2040,Alaska,District 40,President,2338,1377,895


In [109]:
## REORGANIZE DATAFRAMES BASED ON SQL TABLES

In [110]:
## state_unemployment_table

state_unemployment_df2 = state_unemployment_df[['FIPS','Latest Annual Unemployment Rate (2019)','Median Household Income (2018)','% of State Median HH Income']]

state_unemployment = state_unemployment_df2.rename(columns={'FIPS':'fips','Latest Annual Unemployment Rate (2019)':'unemployment_rate_2019','Median Household Income (2018)':'median_household_income_2018','% of State Median HH Income':'percent_of_median_household_income'})

state_unemployment

,fips,unemployment_rate_2019,median_household_income_2018,percent_of_median_household_income
0,1000,3.0,49881.0,1.0
0,2000,6.1,74912.0,1.0
0,4000,4.7,59079.0,1.0
0,5000,3.5,47094.0,1.0
0,6000,4.0,75250.0,1.0
0,8000,2.8,71949.0,1.0
0,9000,3.7,76366.0,1.0
0,10000,3.8,65467.0,1.0
0,11000,5.5,82533.0,1.0
0,12000,3.1,55433.0,1.0


In [111]:
## county_unemployment table

county_unemployment_df2 = county_unemployment_df[['FIPS','Latest Annual Unemployment Rate (2019)','Median Household Income (2018)','% of State Median HH Income']]

county_unemployment = county_unemployment_df2.rename(columns={'FIPS':'fips','Latest Annual Unemployment Rate (2019)':'unemployment_rate_2019','Median Household Income (2018)':'median_household_income_2018','% of State Median HH Income':'percent_of_median_household_income'})

county_unemployment

,fips,unemployment_rate_2019,median_household_income_2018,percent_of_median_household_income
1,1001,2.7,59338.0,1.189591
2,1003,2.7,57588.0,1.154508
3,1005,3.8,34382.0,0.689280
4,1007,3.1,46064.0,0.923478
5,1009,2.7,50412.0,1.010645
...,...,...,...,...
19,56037,3.9,73315.0,1.177581
20,56039,2.7,99087.0,1.591529
21,56041,3.9,63401.0,1.018343
22,56043,3.9,55190.0,0.886458


In [112]:
## voter table

all_parties_cleaned3 = all_parties_cleaned2[['FIPS','office','Democrat Votes','Republican Votes','Other Votes']]

county_voters = all_parties_cleaned3.rename(columns={'FIPS':'fips','Democrat Votes':'democrat_votes','Republican Votes':'republican_votes','Other Votes':'other_votes'})

county_voters

,fips,office,democrat_votes,republican_votes,other_votes
0,1001,President,5936,18172,865
1,1003,President,18458,72883,3874
2,1005,President,4871,5454,144
3,1007,President,1874,6738,207
4,1009,President,2156,22859,573
...,...,...,...,...,...
3177,2037,President,2421,1938,703
3178,2038,President,2758,1143,1194
3179,2039,President,3142,1405,1092
3180,2040,President,2338,1377,895


In [50]:
## Connect to local SQL DB
rds_connection_string = f'{username}:{password}@localhost:5432/ETL_Project_DB'
engine = create_engine(f'postgresql+psycopg2://{rds_connection_string}')

In [51]:
# Read Table Names

engine.table_names()

['state',
 'census_data_state',
 'state_county',
 'census_data_county',
 'covid_state',
 'covid_county',
 'ethnicity_covid',
 'county_unemployment',
 'state_unemployment',
 'county_voters']

### LOAD ALL DATA

In [155]:
# Load state data -- (Aaron's Table)

state = pd.read_csv('Data_Files/state.csv')


state.to_sql(name='state', con=engine, if_exists='append', index=False)

In [156]:
# Load state_county data -- (Aaron's Table)

state_county = pd.read_csv('Data_Files/state_county.csv')

state_county.to_sql(name='state_county', con=engine, if_exists='append', index=False)

In [157]:
# Load State unemployment data -- (Neil's Table)

state_unemployment.to_sql(name='state_unemployment', con=engine, if_exists='append', index=False)

In [158]:
# Load County unemployment data -- (Neil's Table)

county_unemployment.to_sql(name='county_unemployment', con=engine, if_exists='append', index=False)

In [159]:
### ERROR Handling - original county_voters DataFrame above would not load due to foreign key (FIPS) constraint violations (erroneous data)
# Load county_voters data  -- (Neil's Table)
state_county['fips'] = state_county['fips'].astype(str)
county_voters_merge = pd.merge(state_county,county_voters,on='fips',how='left')

county_voters_merge.head()

,fips,abbr,county,latitude,longitude,office,democrat_votes,republican_votes,other_votes
0,1001,AL,Autauga,32.536382,-86.6445,President,5936,18172,865
1,1003,AL,Baldwin,30.659218,-87.7461,President,18458,72883,3874
2,1005,AL,Barbour,31.870670,-85.4055,President,4871,5454,144
3,1007,AL,Bibb,33.015893,-87.1271,President,1874,6738,207
4,1009,AL,Blount,33.977448,-86.5672,President,2156,22859,573


In [160]:
county_voters2 = county_voters_merge[['fips','office','democrat_votes','republican_votes','other_votes']]

In [161]:
county_voters2.to_sql(name='county_voters', con=engine, if_exists='append', index=False)

In [162]:
# Load covid_state data  -- (Aaron's Table) 

covid_state =pd.read_csv('Data_Files/covid_state.csv')

covid_state.to_sql(name='covid_state', con=engine, if_exists='append', index=False)

In [168]:
# Load ethnicity_covid table  -- (Aaron's Table)


# Needed some additional transforming (converting fips to string and dropping null fips rows)
ethnicity_covid = pd.read_csv('Data_Files/ethnicity_covid.csv', dtype = str)
ethnicity_covid = ethnicity_covid.dropna(subset=['fips'])
ethnicity_covid.to_sql(name='ethnicity_covid', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_ethnicity_covid"
DETAIL:  Key (fips)=(1000) already exists.

[SQL: INSERT INTO ethnicity_covid (fips, cases_white, deaths_white, cases_black, deaths_black, "cases_latinX", "deaths_latinX", cases_asian, deaths_asian, cases_total, deaths_total) VALUES (%(fips)s, %(cases_white)s, %(deaths_white)s, %(cases_black)s, %(deaths_black)s, %(cases_latinX)s, %(deaths_latinX)s, %(cases_asian)s, %(deaths_asian)s, %(cases_total)s, %(deaths_total)s)]
[parameters: ({'fips': '1000', 'cases_white': '852', 'deaths_white': '11', 'cases_black': '81', 'deaths_black': '0', 'cases_latinX': None, 'deaths_latinX': None, 'cases_asian': '83', 'deaths_asian': '2', 'cases_total': '2797', 'deaths_total': '22'}, {'fips': '2000', 'cases_white': '25915', 'deaths_white': '779', 'cases_black': '23239', 'deaths_black': '623', 'cases_latinX': None, 'deaths_latinX': None, 'cases_asian': '309', 'deaths_asian': '4', 'cases_tota

In [164]:
# Load census_data_state table -- (Aaron's Table)

census_data_state = pd.read_csv('Data_Files/census_data_state.csv')

census_data_state.to_sql(name='census_data_state', con=engine, if_exists='append', index=False)

In [173]:
# Load census_data_county table -- (Aaron's Table)

census_data_county = pd.read_csv('Data_Files/census_data_county.csv', dtype= str)
## Needed additional cleaning - had to drop duplicate fips rows and remove erroneous fips values

census_data_county = census_data_county.drop_duplicates('fips')
census_data_county_merged = pd.merge(census_data_county,state_county,on='fips',how='right')

census_data_county2 = census_data_county_merged[['fips','land_area','median_household_income','pct_no_health_insurance','pct_households_no_internet']]



In [174]:
census_data_county2.to_sql(name='census_data_county', con=engine, if_exists='append', index=False)

In [60]:
# Extract Tables to CSV (For partner share)

state_unemployment.to_csv('state_unemployment.csv',index=False)

county_unemployment.to_csv('county_unemployment.csv',index=False)

county_voters.to_csv('county_voters.csv',index=False)
